In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

In [3]:
train = pd.read_csv('import_train.csv')
test = pd.read_csv('import_test.csv')

In [4]:
train.head()

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,...,3,4,5,6,7,8,9,>=10,paint_fence,month
0,H14663,2013,1.0,0,1,0,1.0,1,290.0,1,...,0,0,0,0,0,0,0,0,1,12
1,H2037,2015,1.0,0,0,1,0.0,0,490.0,1,...,0,1,0,0,0,0,0,0,0,12
2,H3802,2014,1.0,0,1,0,1.0,1,595.0,1,...,0,0,0,0,0,0,0,0,1,12
3,H3834,2013,1.0,0,0,0,1.0,1,2840.0,1,...,0,0,0,0,0,0,0,0,0,12
4,H5053,2014,1.0,0,0,1,0.0,0,680.0,1,...,1,0,0,0,0,0,0,0,0,12


In [7]:
train.columns

Index(['Customer Id', 'YearOfObservation', 'Insured_Period', 'Residential',
       'Building_Painted', 'Building_Fenced', 'Garden', 'Settlement',
       'Building Dimension', 'Building_Type', 'Date_of_Occupancy', 'Geo_Code',
       'Claim', '   .', '1', '2', '3', '4', '5', '6', '7', '8', '9', '>=10',
       'paint_fence', 'month'],
      dtype='object')

In [8]:
test_id = test['Customer Id']

In [9]:
train = train.drop('Customer Id', axis=1)
test = test.drop('Customer Id', axis=1)

In [10]:
X = train.drop('Claim', axis=1)
y = train.Claim

In [11]:
# Split the data into 30% validation and 70% training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42) #0.15 bank40

In [12]:
from sklearn.metrics import auc

In [13]:
model=CatBoostClassifier(n_estimators=800,eval_metric='AUC',max_depth=5,learning_rate=0.1,od_wait=50, 
                              subsample=0.9,bootstrap_type='Bernoulli',metric_period=20,
                     #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     use_best_model=True)

In [14]:
model.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	test: 0.6697079	best: 0.6697079 (0)	total: 97.1ms	remaining: 1m 17s
20:	test: 0.7127730	best: 0.7137951 (17)	total: 238ms	remaining: 8.81s
40:	test: 0.7150024	best: 0.7150024 (40)	total: 321ms	remaining: 5.95s
60:	test: 0.7124566	best: 0.7150024 (40)	total: 410ms	remaining: 4.96s
80:	test: 0.7100702	best: 0.7150024 (40)	total: 491ms	remaining: 4.36s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7150023676
bestIteration = 40

Shrink model to first 41 iterations.


In [15]:
pred = model.predict_proba(test)[:, 1]

In [16]:
# Split the data into 20% validation and 80% training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42) 

In [17]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb=[]
y_pred_totcb=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit, GroupKFold
fold=StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=CatBoostClassifier(n_estimators=800,eval_metric='AUC',max_depth=4,learning_rate=0.1,#reg_lambda=5,#5
                              subsample=0.9,bootstrap_type='Bernoulli',#leaf_estimation_iterations=10,
                    #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     use_best_model=True)
    #CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=5,learning_rate=0.1,reg_lambda=5,#5
                              #subsample=0.9,bootstrap_type='Bernoulli',
                    #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     #use_best_model=True)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:, 1]
    y_pred_totcb.append(p)

0:	test: 0.6524835	test1: 0.6521008	best: 0.6521008 (0)	total: 9.16ms	remaining: 7.32s
100:	test: 0.7599076	test1: 0.7376466	best: 0.7381347 (96)	total: 371ms	remaining: 2.57s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7381346562
bestIteration = 96

Shrink model to first 97 iterations.
err:  0.7381346561874309
0:	test: 0.6537808	test1: 0.6376939	best: 0.6376939 (0)	total: 4.22ms	remaining: 3.37s
100:	test: 0.7696264	test1: 0.7149817	best: 0.7167299 (77)	total: 370ms	remaining: 2.56s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7167299206
bestIteration = 77

Shrink model to first 78 iterations.
err:  0.7167299206088318
0:	test: 0.6781256	test1: 0.6722213	best: 0.6722213 (0)	total: 3.69ms	remaining: 2.95s
100:	test: 0.7714016	test1: 0.7069831	best: 0.7102185 (43)	total: 369ms	remaining: 2.55s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7102184726
bestIteration = 43

Shrink model to first 44 iterations.
err:  0.710

In [18]:
np.mean(errcb)

0.7216943497881477

In [19]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb1=[]
y_pred_totcb1=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
fold=StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m1=CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=4,learning_rate=0.1,od_wait=50, reg_lambda=3,
                              bootstrap_type='Bayesian',
                     use_best_model=True)
    m1.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m1.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb1.append(roc_auc_score(y_test,preds))
    p1 = m1.predict_proba(test)[:, 1]
    y_pred_totcb1.append(p1)

0:	test: 0.6487806	test1: 0.6601744	best: 0.6601744 (0)	total: 12.8ms	remaining: 12.8s
100:	test: 0.7605327	test1: 0.7342254	best: 0.7350233 (45)	total: 380ms	remaining: 3.38s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7350232595
bestIteration = 45

Shrink model to first 46 iterations.
err:  0.7350232595204654
0:	test: 0.6538945	test1: 0.6460344	best: 0.6460344 (0)	total: 10.3ms	remaining: 10.2s
100:	test: 0.7667430	test1: 0.7137704	best: 0.7156377 (47)	total: 391ms	remaining: 3.48s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7156376695
bestIteration = 47

Shrink model to first 48 iterations.
err:  0.7156376694657781
0:	test: 0.6560630	test1: 0.6437676	best: 0.6437676 (0)	total: 3.44ms	remaining: 3.44s
100:	test: 0.7702210	test1: 0.7096337	best: 0.7104660 (45)	total: 391ms	remaining: 3.48s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7104659657
bestIteration = 45

Shrink model to first 46 iterations.
err:  0.710

In [20]:
np.mean(error1)

0.7203756315628285

In [21]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
fold=KFold(n_splits=5)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m2=XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=800, silent=True, metrics='auc',colsample_bylevel=0.8, reg_alpha=0.8)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m2.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb2.append(roc_auc_score(y_test,preds))
    p2 = m2.predict_proba(test)[:, 1]
    y_pred_totcb2.append(p2)

[0]	validation_0-error:0.196927	validation_1-error:0.219972
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 100 rounds.
[100]	validation_0-error:0.164281	validation_1-error:0.207402
Stopping. Best iteration:
[35]	validation_0-error:0.183485	validation_1-error:0.205307

err:  0.7262499630407143
[0]	validation_0-error:0.200419	validation_1-error:0.218575
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 100 rounds.
[100]	validation_0-error:0.164804	validation_1-error:0.217877
Stopping. Best iteration:
[6]	validation_0-error:0.19588	validation_1-error:0.203212

err:  0.7204686406881426
[0]	validation_0-error:0.207053	validation_1-error:0.197626
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved i

In [22]:
np.mean(error2)

0.7069932155271903

In [23]:
d = {"Customer Id": test_id, 'Claim': np.mean(y_pred_totcb, 0)}
test_predictions = pd.DataFrame(data=d)
test_predictions = test_predictions[["Customer Id", 'Claim']]

In [33]:

test_predictions.to_csv('import_progress.csv', index=False)

In [26]:
test_predictions.head()

,Customer Id,Claim
0,H11920,0.093420
1,H11921,0.083285
2,H9805,0.081117
3,H7493,0.143121
4,H7494,0.117638


In [27]:
d = {"Customer Id": test_id, 'Claim': np.mean(y_pred_totcb1, 0)}
test_predictioned = pd.DataFrame(data=d)
test_predictioned = test_predictioned[["Customer Id", 'Claim']]

In [28]:
test_predictioned.head()

,Customer Id,Claim
0,H11920,0.102572
1,H11921,0.096266
2,H9805,0.080222
3,H7493,0.154024
4,H7494,0.136364


In [29]:
d = {"Customer Id": test_id, 'Claim': np.mean(y_pred_totcb2, 0)}
test_prediction = pd.DataFrame(data=d)
test_prediction = test_prediction[["Customer Id", 'Claim']]

In [30]:
test_predictions.head()

,Customer Id,Claim
0,H11920,0.093420
1,H11921,0.083285
2,H9805,0.081117
3,H7493,0.143121
4,H7494,0.117638


In [36]:
initial = pd.read_csv('initial.csv')

progress = pd.read_csv('import_progress.csv')

0.9736270266770831

0.9683761080837853

In [45]:
test_predictions['Claim'] = (progress['Claim'] * 0.90 + initial['Claim'] * 0.10).round(2)

In [46]:
test_predictions.to_csv('finalstck.csv', index=False)